In [1]:
#Imports
import numpy as np
import pandas as pd
import difflib
import matplotlib.pyplot as plt

In [2]:
transit_agencies = {
    'AC': 'Alameda-Contra Costa Transit District (AC Transit)',
    'AM': 'Capitol Corridor',    
    'AY': 'American Canyon Transit',
    'BA': 'Bay Area Rapid Transit (BART)',
    'CC': 'County Connection (Central Contra Costa Transit Authority)',
    'CE': 'ACE Altamont Corridor Express',
    'CM': 'San Mateo County Transit',
    'DE': 'Dumbarton Express',
    'EM': 'Emery Go-Round',
    'FF': 'Fairfield and Suisun Transit (FAST)',
    'GG': 'Golden Gate Transit',
    'HF': 'Alcatraz Hornblower Ferry',
    'MA': 'Marin Transit',
    'SB': 'San Francisco Bay Ferry',
    'SC': 'Santa Cruz Metro',
    'SF': 'San Francisco Municipal Transportation Agency (SFMTA)',
    'SL': 'SolTrans',
    'SM': 'Stanford Marguerite Shuttle',
    'SO': 'Sonoma County Transit',
    'SR': 'Santa Rosa CityBus',
    'ST': 'SamTrans',
    'TD': 'Tri Delta Transit',
    'VC': 'Vacaville City Coach',
    'VN': 'Vine Transit (Napa County)',
    'VT': 'Santa Clara Valley Transportation Authority (VTA)',
    'WH': 'Livermore Amador Valley Transit Authority',
    'Caltrain': 'Caltrain',
    'PE': 'Petaluma Transit',
    'RV': 'Rio Vista Delta Breeze',
    'UC': 'Union City Transit',
    'WC': 'Western Contra Costa Transit Authority (WestCAT)'
}

In [3]:
#add the new route
FILEPATH = '/Users/cpoliziani/Downloads/Transit Rich/NEW-RESULTS/PLOT/r5-simple-no-local/'
GTFS_folders = ['SF','AC','VT','AY','CC','CE','CM','DE','EM','FF','MA','PE'
                           ,'RV','SB','SC','SL','SM','SO','SR','ST','TD','UC',
                'VC','VN','VT','WC', 'WH','BA','Caltrain','AM','HF','GG'
                           ]
shapes_df = pd.DataFrame()
trip_ID = []
lons = []
lats = []
agency = []
i = 0
for gtfs in GTFS_folders:
    trips = pd.read_csv(FILEPATH+gtfs+'/trips.txt')
    stop_times = pd.read_csv(FILEPATH+gtfs+'/stop_times.txt')
    stops = pd.read_csv(FILEPATH+gtfs+'/stops.txt')
    stopid_to_lat = dict(zip(stops['stop_id'], stops['stop_lat']))
    stopid_to_lon = dict(zip(stops['stop_id'], stops['stop_lon']))

    calendar = pd.read_csv(FILEPATH+gtfs+'/calendar.txt')
    
    valid_calendar_ids = calendar['service_id'][(calendar['friday']==1)&(calendar['start_date']<=20170922)&(calendar['end_date']>=20170922)].to_list()
    trips = trips[trips['service_id'].isin(valid_calendar_ids)]
    print(len(trips))
    
    try:
        shapes = pd.read_csv(FILEPATH+gtfs+'/shapes.txt')
        is_shape = True
        
    except:
        print('no shapes for',gtfs)
        is_shape = False
    
    if is_shape:
        for shape in trips['shape_id']:
            shapes_trip = shapes[shapes['shape_id']==shape]
            for lat, lon in zip(shapes_trip['shape_pt_lat'],shapes_trip['shape_pt_lon']):
                lats.append(lat)
                lons.append(lon)
                trip_ID.append(i)
                agency.append(transit_agencies[gtfs])
            i+=1
    else:
        for trip in trips['trip_id']:
            stop_times_trip = stop_times[stop_times['trip_id']==trip]
            for stop in stop_times_trip['stop_id']:
                lats.append(stopid_to_lat[stop])
                lons.append(stopid_to_lon[stop])
                trip_ID.append(i)
                agency.append(transit_agencies[gtfs])
            i+=1

shapes_df['TRIP'] = trip_ID     
shapes_df['LAT'] = lats     
shapes_df['LON'] = lons     
shapes_df['AGENCY'] = agency     
    
    
    


11881
6715
4028
5
no shapes for AY
921
8
257
100
243
520
653
183
10
36
5041
549
804
313
384
1932
471
281
130
235
4028
633
554
1101
92
57
27
1854


In [4]:
shapes_df.to_csv('shapes_table.csv')